In [1]:
import pandas as pd
import requests
import pymongo
from sqlalchemy import create_engine



url = 'https://data.cityofchicago.org/api/views/5udb-dr6f/rows.json?accessType=DOWNLOAD'
r= requests.get(url).json()


In [2]:
df = pd.DataFrame(r['data'])
df = df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=False)
df =df.dropna(axis=1, how='all', thresh=None, subset=None, inplace=False)
df.rename(columns={8:'Name',9:'DBA',10:'identifier', 11:'Type',12:'Risk', 13:'Street',14:'City', 15:'State', 16:'Zip', 17:'Date',18:'Inspection',19:'Success',20:'Description', 21:'lat', 22:'lng'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date']).apply(lambda x:x.strftime('%m/%d/%Y'))
df.head(5)

,Name,DBA,identifier,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,lat,lng
0,FULTON GALLEY,None,2636278,Restaurant,Risk 3 (Low),1115 W FULTON MARKET,CHICAGO,IL,60607,06/07/2019,License Re-Inspection,Pass w/ Conditions,None,41.88657130932981,-87.65517447025061
1,WALDORF ASTORIA CHICAGO,MARGEAUX/ CAFE/EMPLOYEE CAFE,2134586,Restaurant,Risk 1 (High),11 E WALTON ST,CHICAGO,IL,60611,02/14/2012,License,Pass,None,41.89983492074242,-87.6277991723728
2,LAS FRESAS,LAS FRESAS,2247279,Restaurant,Risk 2 (Medium),7057 N CLARK ST,CHICAGO,IL,60626,06/06/2019,Canvass,Out of Business,None,42.010675486099814,-87.67416668417441
3,GORILLA SUSHI LOGAN SQUARE,GORILLA SUSHI LOGAN SQUARE,2543141,Restaurant,Risk 1 (High),2735 N MILWAUKEE AVE,CHICAGO,IL,60647,06/06/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.93096908810037,-87.71028632230771
4,PICNIC WINE & PROVISIONS,PICNIC WINE & PROVISIONS,2636701,Restaurant,Risk 2 (Medium),7301 N SHERIDAN RD,CHICAGO,IL,60626,06/06/2019,License,Fail,None,42.0140411322766,-87.6639709089456


In [112]:
###########Method 1 MONGO DB   #################

#insert into Mongo
#REMEMBER TO START MONGOD in your bash!!!!!!!!
conn = 'mongodb://localhost:27017'
client=pymongo.MongoClient(conn)
#define DB and collection/table to work with
db = client.ChicagoRest
#convert DataFrame to JSON format for insert
#Load all dataframes
records = json.loads(df.to_json(orient='records'))
db.CompanyData.insert_many(records)

In [3]:
#sqlite
connection_string = "sqlite:///ChicagoRestaurants.sqlite"
engine = create_engine(connection_string)
#list tables names- if you want
#engine.table_names()

####Create the database to match our columns in our dataframe####
#Name	DBA	ID	Type	Risk	Street	City	State	Zip	Date	Inspection	Success	Description	lat	lng


##############IMPORTANT###############################
########### hash out this after creating as it will give an error stating the DB already exists after running the first time ####
####apparently the line below is not needed to create the sql lite.. if unhash it creates another table in the db
engine.execute("CREATE TABLE RestaurantData(id INT PRIMARY KEY, Name TEXT, DBA TEXT, identifier TEXT, Type TEXT, Risk TEXT, Street TEXT, City TEXT, State TEXT, Zip TEXT, Date TEXT, Inspection TEXT, Success TEXT, Description TEXT, Lat Int, Lng Int);")
#engine.execute("CREATE TABLE RestaurantData(Name TEXT, DBA TEXT, identifier TEXT, Type TEXT, Risk TEXT, Street TEXT, City TEXT, State TEXT, Zip TEXT, Date TEXT, Inspection TEXT, Success TEXT, Description TEXT, Lat Int, Lng Int);")

################################################################################################################

####### PLUG that data into SQLITE#####
df.to_sql(name="RestaurantData", con=engine, if_exists="append", index=False)
#####SHOW ME THE MONEY#####
pd.read_sql_query("SELECT * FROM RestaurantData",con=engine).head()

,id,Name,DBA,identifier,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,Lat,Lng
0,None,FULTON GALLEY,None,2636278,Restaurant,Risk 3 (Low),1115 W FULTON MARKET,CHICAGO,IL,60607,06/07/2019,License Re-Inspection,Pass w/ Conditions,None,41.886571,-87.655174
1,None,WALDORF ASTORIA CHICAGO,MARGEAUX/ CAFE/EMPLOYEE CAFE,2134586,Restaurant,Risk 1 (High),11 E WALTON ST,CHICAGO,IL,60611,02/14/2012,License,Pass,None,41.899835,-87.627799
2,None,LAS FRESAS,LAS FRESAS,2247279,Restaurant,Risk 2 (Medium),7057 N CLARK ST,CHICAGO,IL,60626,06/06/2019,Canvass,Out of Business,None,42.010675,-87.674167
3,None,GORILLA SUSHI LOGAN SQUARE,GORILLA SUSHI LOGAN SQUARE,2543141,Restaurant,Risk 1 (High),2735 N MILWAUKEE AVE,CHICAGO,IL,60647,06/06/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.930969,-87.710286
4,None,PICNIC WINE & PROVISIONS,PICNIC WINE & PROVISIONS,2636701,Restaurant,Risk 2 (Medium),7301 N SHERIDAN RD,CHICAGO,IL,60626,06/06/2019,License,Fail,None,42.014041,-87.663971


In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [6]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///ChicagoRestaurants.sqlite")

In [7]:
########################################################################################################
######################QUERY 1. THIS PULLS ALL DATA AND RETURNS JSONIFIED################################
########################################################################################################
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
restaurants = Base.classes.RestaurantData
session = Session(engine)
###Query####
Output = []
#Name	DBA	identifier	Type	Risk	Street	City	State	Zip	Date	Inspection	Success	Description	Lat	Lng
for row in session.query(restaurants, restaurants.Name, restaurants.DBA, restaurants.Type,restaurants.Risk,restaurants.Street, restaurants.City, \
                         restaurants.State, restaurants.Zip, restaurants.Date,restaurants.Inspection, restaurants.Success, restaurants.Description, \
                        restaurants.Lat, restaurants.Lng).all():
    Output.append(row)
df = pd.DataFrame(data=Output)
df.to_json(orient='records')

In [ ]:
########################################################################################################
######################QUERY 2. THIS PULLS ALL DATA BY YEAR################################
########################################################################################################